# パッケージの読み込み

In [43]:
import os
# 数値処理
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, cross_validate
from sklearn.preprocessing import PolynomialFeatures

# グラフ描画
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Meiryo'

# 学習モデル
import optuna
from lightgbm import LGBMClassifier
import optuna.integration.lightgbm as lgb 

# データの読み込み

In [33]:
# データの配置場所
output_dir = 'C:/Users/rhira/Documents/GitHub'\
             '/signate-competition-Liver-disease-beginner/features'

# ダミー変数化のみ
train_dummied = pd.read_feather(os.path.join(output_dir, 'train_dummied_feature.feather'))
test_dummied = pd.read_feather(os.path.join(output_dir, 'test_dummied_feature.feather'))
# 2次の交互作用まで
train_quadratic = pd.read_feather(os.path.join(output_dir, 'train_qiadratic_feature.feather'))
test_quadratic = pd.read_feather(os.path.join(output_dir, 'test_quadratic_feature.feather'))
# 3次の交互作用まで
train_cubic = pd.read_feather(os.path.join(output_dir, 'train_qiadratic_feature.feather'))
test_cubic = pd.read_feather(os.path.join(output_dir, 'test_quadratic_feature.feather'))

# ターゲット
train_target = pd.read_feather(os.path.join(output_dir, 'train_target.feather'))

# optuna用の関数の作成

In [37]:
def objectives(trial):
    # scikit-learnでお試しデータの準備
    X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state=0, stratify=target)

    # optunaでのハイパーパラメータサーチ範囲の設定
    params = {
        #fixed
        'boosting_type':'gbdt',
        'max_depth':-1,
        'learning_rate':0.1,
        'n_estimators': 1000,
        'metric':'l2',

        #variable
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'reg_alpha': trial.suggest_loguniform('reg_alpha',0.001, 10),
        'reg_lambda':trial.suggest_loguniform('reg_lambda', 0.001, 10),

    }

    # LightGBMで学習+予測
    model = LGBMClassifier(**params,random_state=0)# 追加部分
    
    # kFold交差検定で決定係数を算出し、各セットの平均値を返す
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    scores = cross_validate(model, X=X_train, y=y_train,cv=kf)   

    return scores['test_score'].mean()

# ダミー変数化データの学習

In [38]:
# 使用する特徴量とターゲット
feature = train_dummied
target = train_target
# optunaによる最適化呼び出し
opt = optuna.create_study(direction='maximize')
opt.optimize(objectives, n_trials=100)


# 実行結果表示
print('最終トライアル回数:{}'.format(len(opt.trials)))
print('ベストトライアル:')
trial = opt.best_trial
print('値:{}'.format(trial.value))
print('パラメータ:')
for key, value in trial.params.items():
    print('{}:{}'.format(key, value))

[I 2020-09-21 19:09:53,861] A new study created in memory with name: no-name-b8974b8d-1eab-485d-b95e-9eefde9352ff
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:09:54,345] Trial 0 finished with value: 0.8509315853190421 and parameters: {'num_leaves': 73, 'reg_alpha': 0.03446794399577971, 'reg_lambda': 0.004137798550166451}. Best is trial 0 with value: 0.8509315853190421.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:09:54,726] Trial 1 finished with value: 0.8619452564443263 and parameters: {'num_leaves': 142, 'reg_alpha': 0.7142415

[I 2020-09-21 19:10:04,295] Trial 16 finished with value: 0.8572208934951427 and parameters: {'num_leaves': 253, 'reg_alpha': 0.31408875101280953, 'reg_lambda': 1.7873953041025017}. Best is trial 11 with value: 0.8713275459887205.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:10:04,611] Trial 17 finished with value: 0.8666179466737532 and parameters: {'num_leaves': 213, 'reg_alpha': 3.22095044199463, 'reg_lambda': 0.03795977637984466}. Best is trial 11 with value: 0.8713275459887205.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:1

[I 2020-09-21 19:10:12,682] Trial 33 finished with value: 0.8650677650810524 and parameters: {'num_leaves': 275, 'reg_alpha': 5.0618503823112535, 'reg_lambda': 0.49370242188190744}. Best is trial 32 with value: 0.8760519089379041.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:10:12,997] Trial 34 finished with value: 0.8634659107685948 and parameters: {'num_leaves': 285, 'reg_alpha': 9.909488062966789, 'reg_lambda': 9.479714797516323}. Best is trial 32 with value: 0.8760519089379041.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:10

[I 2020-09-21 19:10:20,979] Trial 50 finished with value: 0.8697552189446953 and parameters: {'num_leaves': 253, 'reg_alpha': 8.014985536791103, 'reg_lambda': 9.835657754106549}. Best is trial 32 with value: 0.8760519089379041.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:10:21,333] Trial 51 finished with value: 0.8729072548498538 and parameters: {'num_leaves': 248, 'reg_alpha': 7.685754667565908, 'reg_lambda': 8.13649873389338}. Best is trial 32 with value: 0.8760519089379041.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:10:21,

[I 2020-09-21 19:10:30,180] Trial 67 finished with value: 0.8619009655416777 and parameters: {'num_leaves': 261, 'reg_alpha': 3.1153211523140345, 'reg_lambda': 3.214238210210639}. Best is trial 32 with value: 0.8760519089379041.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:10:30,688] Trial 68 finished with value: 0.8603507839489768 and parameters: {'num_leaves': 278, 'reg_alpha': 1.5005368118367284, 'reg_lambda': 4.749512423345049}. Best is trial 32 with value: 0.8760519089379041.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:10:

[I 2020-09-21 19:10:37,212] Trial 84 finished with value: 0.8635028198541352 and parameters: {'num_leaves': 278, 'reg_alpha': 5.661589117468053, 'reg_lambda': 0.001933418054384553}. Best is trial 32 with value: 0.8760519089379041.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:10:37,632] Trial 85 finished with value: 0.8650677650810522 and parameters: {'num_leaves': 242, 'reg_alpha': 1.9839898789729857, 'reg_lambda': 4.096719383636952}. Best is trial 32 with value: 0.8760519089379041.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:1

最終トライアル回数:100
ベストトライアル:
値:0.8760519089379041
パラメータ:
num_leaves:284
reg_alpha:5.366085059893228
reg_lambda:4.137003485200064


# 2次の交互作用データの学習

In [34]:
# 使用する特徴量とターゲット
feature = train_quadratic
target = train_target
# optunaによる最適化呼び出し
opt = optuna.create_study(direction='maximize')
opt.optimize(objectives, n_trials=100)


# 実行結果表示
print('最終トライアル回数:{}'.format(len(opt.trials)))
print('ベストトライアル:')
trial = opt.best_trial
print('値:{}'.format(trial.value))
print('パラメータ:')
for key, value in trial.params.items():
    print('{}:{}'.format(key, value))

[I 2020-09-21 19:04:25,009] A new study created in memory with name: no-name-c2a8c9d7-2172-42da-8e6c-9d5fad5ff10e
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:25,756] Trial 0 finished with value: 0.8414976230548912 and parameters: {'num_leaves': 197, 'reg_alpha': 0.007457818925212517, 'reg_lambda': 0.002783503733138495}. Best is trial 0 with value: 0.8414976230548912.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:26,214] Trial 1 finished with value: 0.8650825287152685 and parameters: {'num_leaves': 36, 'reg_alpha': 3.143948

[I 2020-09-21 19:04:40,260] Trial 16 finished with value: 0.8414976230548912 and parameters: {'num_leaves': 35, 'reg_alpha': 0.0013163714277321084, 'reg_lambda': 5.46599698856516}. Best is trial 1 with value: 0.8650825287152685.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:40,709] Trial 17 finished with value: 0.8477869312309917 and parameters: {'num_leaves': 117, 'reg_alpha': 1.9461329588450311, 'reg_lambda': 0.18297082268110496}. Best is trial 1 with value: 0.8650825287152685.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04

[I 2020-09-21 19:04:51,510] Trial 33 finished with value: 0.8430625682818081 and parameters: {'num_leaves': 205, 'reg_alpha': 1.001566408973989, 'reg_lambda': 5.438052823022747}. Best is trial 1 with value: 0.8650825287152685.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:52,051] Trial 34 finished with value: 0.8587784569049518 and parameters: {'num_leaves': 296, 'reg_alpha': 2.839154546638899, 'reg_lambda': 2.6103374761942066}. Best is trial 1 with value: 0.8650825287152685.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:04:52,

[I 2020-09-21 19:05:01,423] Trial 50 finished with value: 0.8525186759972835 and parameters: {'num_leaves': 212, 'reg_alpha': 2.0067545752169247, 'reg_lambda': 9.963867911121994}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:01,879] Trial 51 finished with value: 0.861923110993002 and parameters: {'num_leaves': 192, 'reg_alpha': 3.171728283564597, 'reg_lambda': 1.7706900142454602}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:0

[I 2020-09-21 19:05:13,257] Trial 67 finished with value: 0.8666253284908613 and parameters: {'num_leaves': 38, 'reg_alpha': 3.5890861655707598, 'reg_lambda': 9.823447792546018}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:14,068] Trial 68 finished with value: 0.8446496589600496 and parameters: {'num_leaves': 32, 'reg_alpha': 1.3061345366082056, 'reg_lambda': 9.851410759778924}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:14

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2020-09-21 19:05:14,972] Trial 70 finished with value: 0.8603507839489769 and parameters: {'num_leaves': 31, 'reg_alpha': 3.2476599042029584, 'reg_lambda': 0.49599746756897073}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:15,661] Trial 71 finished with value: 0.8650603832639442 and parameters: {'num_leaves': 49, 'reg_alpha': 4.1138440539286405, 'reg_lambda': 6.046537056632454}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:

[I 2020-09-21 19:05:24,593] Trial 87 finished with value: 0.8634880562199191 and parameters: {'num_leaves': 200, 'reg_alpha': 7.620978256678386, 'reg_lambda': 0.8500455771147704}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:25,039] Trial 88 finished with value: 0.861923110993002 and parameters: {'num_leaves': 262, 'reg_alpha': 5.7683065252258094, 'reg_lambda': 1.4581429837775013}. Best is trial 43 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:05:

最終トライアル回数:100
ベストトライアル:
値:0.8666327103079694
パラメータ:
num_leaves:189
reg_alpha:3.029104937419662
reg_lambda:2.319511005423461


# 3次の交互作用データの学習

In [35]:
# 使用する特徴量とターゲット
feature = train_cubic
target = train_target
# optunaによる最適化呼び出し
opt = optuna.create_study(direction='maximize')
opt.optimize(objectives, n_trials=100)


# 実行結果表示
print('最終トライアル回数:{}'.format(len(opt.trials)))
print('ベストトライアル:')
trial = opt.best_trial
print('値:{}'.format(trial.value))
print('パラメータ:')
for key, value in trial.params.items():
    print('{}:{}'.format(key, value))

[I 2020-09-21 19:06:04,033] A new study created in memory with name: no-name-b08385f1-fee4-49cf-863c-d954bf4700de
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:04,652] Trial 0 finished with value: 0.8383751144181651 and parameters: {'num_leaves': 173, 'reg_alpha': 0.05916538210413745, 'reg_lambda': 0.0025488044873853287}. Best is trial 0 with value: 0.8383751144181651.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:05,377] Trial 1 finished with value: 0.8430404228304841 and parameters: {'num_leaves': 19, 'reg_alpha': 0.007223

[I 2020-09-21 19:06:16,409] Trial 16 finished with value: 0.8540688575899843 and parameters: {'num_leaves': 135, 'reg_alpha': 2.394640961734748, 'reg_lambda': 0.053698555204126495}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:16,972] Trial 17 finished with value: 0.8430773319160245 and parameters: {'num_leaves': 74, 'reg_alpha': 1.1618150173431718, 'reg_lambda': 2.260239220687913}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:1

[I 2020-09-21 19:06:25,948] Trial 33 finished with value: 0.8587784569049517 and parameters: {'num_leaves': 68, 'reg_alpha': 5.566511081473738, 'reg_lambda': 0.23522333814414323}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:26,446] Trial 34 finished with value: 0.8446570407771578 and parameters: {'num_leaves': 32, 'reg_alpha': 1.5667570262011166, 'reg_lambda': 0.050782693196799504}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:

[I 2020-09-21 19:06:35,691] Trial 50 finished with value: 0.8430699500989164 and parameters: {'num_leaves': 57, 'reg_alpha': 0.04375164264869748, 'reg_lambda': 0.47241406497358784}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:36,073] Trial 51 finished with value: 0.8635028198541352 and parameters: {'num_leaves': 36, 'reg_alpha': 6.415930891924241, 'reg_lambda': 1.0796157152099368}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:3

[I 2020-09-21 19:06:43,235] Trial 67 finished with value: 0.8509315853190422 and parameters: {'num_leaves': 54, 'reg_alpha': 2.005558664949153, 'reg_lambda': 2.849158432942903}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:43,667] Trial 68 finished with value: 0.8634954380370273 and parameters: {'num_leaves': 32, 'reg_alpha': 4.561538899025011, 'reg_lambda': 0.41082725621851596}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:44,0

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


[I 2020-09-21 19:06:44,543] Trial 70 finished with value: 0.8556411846340094 and parameters: {'num_leaves': 31, 'reg_alpha': 2.934332204756724, 'reg_lambda': 1.5409207957617306}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:44,956] Trial 71 finished with value: 0.8634880562199191 and parameters: {'num_leaves': 33, 'reg_alpha': 6.14491588768861, 'reg_lambda': 0.1846028861765985}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:45,37

[I 2020-09-21 19:06:53,250] Trial 87 finished with value: 0.8556559482682257 and parameters: {'num_leaves': 16, 'reg_alpha': 3.230111632006935, 'reg_lambda': 0.5306993926161488}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:53,900] Trial 88 finished with value: 0.8430625682818081 and parameters: {'num_leaves': 147, 'reg_alpha': 1.1670159625463137, 'reg_lambda': 1.2184644234662159}. Best is trial 8 with value: 0.8666327103079694.
C:\Users\rhira\Anaconda3\envs\env\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
[I 2020-09-21 19:06:54

最終トライアル回数:100
ベストトライアル:
値:0.8681976555348866
パラメータ:
num_leaves:21
reg_alpha:3.917236543553505
reg_lambda:0.3417223854757551


In [67]:
# 使用する特徴量とターゲット
feature = train_quadratic
target = train_target

X_train, X_test, y_train, y_test = train_test_split(feature, target, random_state=0, stratify=target)
lgb_params = {'objective': 'binary',
              'metric': 'auc', #コンペ指定の評価指標がAUC
              #'class_weight': 'balanced' #ここは不要でした
             }

lgb_train = lgb.Dataset(X_train, y_train) #学習用
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train) #Boosting用
model = lgb.train(lgb_params, lgb_train, valid_sets=lgb_eval,
                      num_boost_round=1000,
                      early_stopping_rounds=20,
                      verbose_eval=10)

[I 2020-09-21 19:31:19,003] A new study created in memory with name: no-name-f4fd80e3-c7f0-4575-9fa7-c3f68d423d4e

feature_fraction, val_score: -inf:   0%|                                                         | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001953 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12608
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 65
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes


feature_fraction, val_score: 0.939072:  14%|######4                                      | 1/7 [00:00<00:01,  3.41it/s][I 2020-09-21 19:31:19,305] Trial 0 finished with value: 0.9390722569134701 and parameters: {'feature_fraction': 0.7}. Best is trial 0 with value: 0.9390722569134701.

feature_fraction, val_score: 0.939072:  14%|######4                                      | 1/7 [00:00<00:01,  3.41it/s]

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002023 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12608
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai


feature_fraction, val_score: 0.939072:  29%|############8                                | 2/7 [00:00<00:01,  3.09it/s][I 2020-09-21 19:31:19,701] Trial 1 finished with value: 0.9372881355932203 and parameters: {'feature_fraction': 1.0}. Best is trial 0 with value: 0.9390722569134701.

feature_fraction, val_score: 0.939072:  29%|############8                                | 2/7 [00:00<00:01,  3.09it/s]

...


min_data_in_leaf, val_score: 0.947101:  60%|###########################                  | 3/5 [00:00<00:00,  3.14it/s][I 2020-09-21 19:31:40,204] Trial 65 finished with value: 0.9313113291703836 and parameters: {'min_child_samples': 5}. Best is trial 63 with value: 0.9450490633363069.

min_data_in_leaf, val_score: 0.947101:  60%|###########################                  | 3/5 [00:00<00:00,  3.14it/s]

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001632 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12608
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[10]	valid_0's auc: 0.936039
[20]	valid_0's auc: 0.930508
[30]	valid_0's auc: 0.934701
Early stopping, best iteration is:
[10]	valid_0's auc: 0.936039



min_data_in_leaf, val_score: 0.947101:  80%|####################################         | 4/5 [00:01<00:00,  3.93it/s][I 2020-09-21 19:31:40,308] Trial 66 finished with value: 0.9360392506690455 and parameters: {'min_child_samples': 10}. Best is trial 63 with value: 0.9450490633363069.

min_data_in_leaf, val_score: 0.947101:  80%|####################################         | 4/5 [00:01<00:00,  3.93it/s]

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 283, number of negative: 354
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001959 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12608
[LightGBM] [Info] Number of data points in the train set: 637, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.444270 -> initscore=-0.223850
[LightGBM] [Info] Start training from score -0.223850
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai


min_data_in_leaf, val_score: 0.947101: 100%|#############################################| 5/5 [00:01<00:00,  3.80it/s]


In [70]:
pred = model.predict(test_quadratic) #口座申込み確率値

In [72]:
pd.DataFrame(np.round(pred)).to_csv('lightgbm_quadratic_2020_09_21_19_33.csv', header=None)

In [71]:
pred

array([0.00470672, 0.34961165, 0.00592793, 0.02101915, 0.01534845,
       0.01080333, 0.99435336, 0.4053496 , 0.69588352, 0.11716483,
       0.07780537, 0.021814  , 0.0911902 , 0.99455429, 0.00578151,
       0.1210198 , 0.96067844, 0.04072305, 0.01523303, 0.00576643,
       0.43049568, 0.01231138, 0.04461645, 0.37874193, 0.10197774,
       0.03666379, 0.03807565, 0.99142837, 0.66197237, 0.99496593,
       0.09393412, 0.98625123, 0.1111846 , 0.51765304, 0.94252525,
       0.99082639, 0.00574451, 0.31828785, 0.00502886, 0.99186931,
       0.69227117, 0.73693189, 0.98167142, 0.41267614, 0.23283835,
       0.17167715, 0.00476199, 0.98364624, 0.55090347, 0.51231062,
       0.31123339, 0.75961864, 0.98371038, 0.61676754, 0.64103163,
       0.00598056, 0.59675399, 0.02875519, 0.99194219, 0.03077027,
       0.72396207, 0.01525223, 0.99488705, 0.24951071, 0.04336833,
       0.37233106, 0.98606223, 0.9820709 , 0.13210544, 0.00805921,
       0.97560424, 0.19621625, 0.10570819, 0.2977032 , 0.01502